weaves

This is for Revolut. Part II of Challenge

This uses the output of anal2.q to predict an unengaged user.
Analysis uses a simplified users table.

In [68]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer

from sklearn import svm
from sklearn import datasets

%load_ext autoreload
%autoreload 2

pd.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'0.24.2'

In [108]:
# Read the CSV. This has been written out by q/kdb+. Booleans were converted by q/kdb+
# The userid is the index.
df0 = pd.read_csv('cache/out/xusers1.csv', index_col=0)

In [109]:
df0.head()

,ctry,uscrypt0,plan,push1,email1,ncontacts,age0,lost0,nico0,nicu0,...,tdecl,tfail,tpend,trvrt,inb,brand,v,nactive,inactive,inactive1
userid,,,,,,,,,,,,,,,,,,,,,
0,PL,True,STANDARD,1,1,3,30,False,False,False,...,True,True,False,True,True,Apple,36.14,False,False,False
1,GB,False,STANDARD,-1,-1,21,44,False,True,True,...,True,False,False,True,True,Apple,50.05,False,False,False
2,PL,False,STANDARD,0,0,21,32,False,False,False,...,True,True,False,True,True,Android,12.66,False,False,False
3,FR,False,STANDARD,1,0,0,25,False,False,False,...,False,False,True,True,True,Apple,64.57,False,False,False
4,GB,False,STANDARD,-1,-1,2,34,False,False,False,...,False,False,False,True,True,Apple,203.99,False,False,False


In [110]:
# Most fields are string
df0.columns.to_series().groupby(df0.dtypes).groups

{dtype('bool'): Index(['uscrypt0', 'lost0', 'nico0', 'nicu0', 'nifpco0', 'otau0', 'pf0',
        'tcncl', 'tdecl', 'tfail', 'tpend', 'trvrt', 'inb', 'nactive',
        'inactive', 'inactive1'],
       dtype='object'),
 dtype('int64'): Index(['push1', 'email1', 'ncontacts', 'age0'], dtype='object'),
 dtype('float64'): Index(['v'], dtype='object'),
 dtype('O'): Index(['ctry', 'plan', 'brand'], dtype='object')}

## Outcomes: 3 types of unengagedness

There are three types of unengaged user, where a user is one that has no activity

- at all. Labelled as nactive. No transactions from 2018.01.01. Do not use the average value of transactions as a feature it's always zero.
- in the last year. Labelled as inactive. Last activity was over a year ago. The average value is 0 for 60% so not too useful
- over a month ago. Labelled as inactive1. Average value is 0 for 30% of this population.

In [111]:
df0[df0.select_dtypes(['object']).columns] = df0.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [112]:
# The groups phrase doesn't work for category
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19430 entries, 0 to 19429
Data columns (total 24 columns):
ctry         19430 non-null category
uscrypt0     19430 non-null bool
plan         19430 non-null category
push1        19430 non-null int64
email1       19430 non-null int64
ncontacts    19430 non-null int64
age0         19430 non-null int64
lost0        19430 non-null bool
nico0        19430 non-null bool
nicu0        19430 non-null bool
nifpco0      19430 non-null bool
otau0        19430 non-null bool
pf0          19430 non-null bool
tcncl        19430 non-null bool
tdecl        19430 non-null bool
tfail        19430 non-null bool
tpend        19430 non-null bool
trvrt        19430 non-null bool
inb          19430 non-null bool
brand        19430 non-null category
v            19430 non-null float64
nactive      19430 non-null bool
inactive     19430 non-null bool
inactive1    19430 non-null bool
dtypes: bool(16), category(3), float64(1), int64(4)
memory usage: 1.2 MB


In [113]:
# Store the categories
cats0 = dict([ (x, tuple(df0[x].cat.categories)) for x in df0.select_dtypes(['category']).columns ])
cats0

{'ctry': ('AT',
  'AU',
  'BE',
  'BG',
  'CH',
  'CY',
  'CZ',
  'DE',
  'DK',
  'EE',
  'ES',
  'FI',
  'FR',
  'GB',
  'GF',
  'GG',
  'GI',
  'GP',
  'GR',
  'HR',
  'HU',
  'IE',
  'IM',
  'IS',
  'IT',
  'JE',
  'LI',
  'LT',
  'LU',
  'LV',
  'MQ',
  'MT',
  'NL',
  'NO',
  'PL',
  'PT',
  'RE',
  'RO',
  'SE',
  'SI',
  'SK'),
 'plan': ('METAL',
  'METAL_FREE',
  'PREMIUM',
  'PREMIUM_FREE',
  'PREMIUM_OFFER',
  'STANDARD'),
 'brand': ('Android', 'Apple', 'Unknown')}

### Reload the dataset for the model

You need to run this after you have chosen and evaluated.

In [129]:
# Make a copy and quantify the categories by using the category codes.
df2 = df0.copy(deep=True)
df2[df2.select_dtypes(['category']).columns] = df2.select_dtypes(['category']).apply(lambda x: x.cat.codes)
if 'y' in globals():
    del globals()['y']

### Case 1: Never active

Evaluate one of these data cases - that is, re-evaluate the cell above and then Case 1 *or* Case 2 *or* Case 3.

This case Remove value, and inbound transactions

In [130]:
# And obtain the outcome as y. And the features as X, drop columns and rows that would confuse the model.

y = df2.nactive
df2.drop(columns=['nactive', 'inactive', 'inactive1', 'v', 'inb'], inplace=True)
# Note: this uses the values property to get the underlying matrix.
X = df2.values

### Case 2: Inactive

Remove value (for now), but keep inb.
Remove nactive samples.

In [82]:
df2 = df2[~df2['nactive']]
if not 'y' in globals():
    y = df2.inactive
    df2.drop(columns=['inactive', 'nactive', 'inactive1', 'v'], inplace=True)
    X = df2.values

### Case 3: Inactive

Keep value and inb

In [124]:
# And obtain the outcome as y. And the features as X
# Note: this uses the values property to get the underlying matrix.
df2 = df2[~df2['nactive']]
df2 = df2[~df2['inactive']]
if not 'y' in globals():
    y = df2.inactive1
    df2.drop(columns=['inactive1', 'nactive', 'inactive1'], inplace=True)
    X = df2.values

KeyError: "['inactive1userid'] not found in axis"

## Model cases

Again choose one to apply to the data you've chosen.

These are a couple of models to evaluate.

The first SVC is not practical for "big" data (width and count), but works well for smaller ones.

The Multi-Layer Perceptron Classifier is a useful neural network and can get reasonable results quickly - it parallizes well. The hidden_layer_sizes need tuning to get a cross validation score over 0.5.

The logistic regression is a very solid performer.

In [131]:
clf = svm.SVC(kernel='linear', C=1)

In [37]:
clf =  MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(X.shape[1], X.shape[1], 2), 
                     random_state=1, warm_start=True)

In [29]:
clf = LogisticRegression(C=1e5, solver='lbfgs', multi_class='auto')

## From cross-validation results, choose the most accurate model


To evaluate the model under a cross-validation scheme, run this cell.

In [132]:
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.96861333, 0.98018528, 0.97915594, 0.97941328, 0.98069498])